<a href="https://colab.research.google.com/github/null-buster/speech-to-text/blob/main/Speaker_Diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Resemblyzer


In [ ]:
#Code to convert file to .wav
!pip install pydub

In [ ]:
!pip3 install spectralcluster

In [ ]:
from pydub import AudioSegment

def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.split('.')[0] + '.wav'
    sound.export(audio_file_path, format="wav")
    return audio_file_path

audio_file_path = mp3_to_wav('/content/test audio.mp3')
print(audio_file_path)

/content/test audio.wav


In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = '/content/test audio.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.03 seconds.
(1160, 256)


In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)
print(labels)

[0 0 0 ... 0 0 0]


In [ ]:
from resemblyzer.audio import sampling_rate

def create_labelling(labels,wav_splits):
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)
print(labelling)


[('0', 0, 6.08), ('1', 6.08, 7.64), ('0', 7.64, 9.02), ('1', 9.02, 11.42), ('0', 11.42, 23.24), ('1', 23.24, 24.08), ('0', 24.08, 26.06), ('1', 26.06, 26.18), ('0', 26.18, 44.42), ('1', 44.42, 45.08), ('0', 45.08, 47.96), ('1', 47.96, 48.38), ('0', 48.38, 50.48), ('1', 50.48, 50.54), ('0', 50.54, 50.6), ('1', 50.6, 50.9), ('0', 50.9, 58.04), ('1', 58.04, 59.72), ('0', 59.72, 61.22), ('1', 61.22, 62.72), ('0', 62.72, 70.34)]
